In [81]:
#import libraries
import pandas as pd
import snowflake.connector
import numpy as np
import openai
import os
import json

# Read openai api key from file apikey.txt
api_key_file = 'apikey.txt'
if os.path.isfile(api_key_file):
    with open(api_key_file) as f:
        openai.api_key = f.readline()
else:
    print(f"Error: {api_key_file} not found.")

OPENAI_API_KEY = openai.api_key
content = ""

#login to snowflake db
con = snowflake.connector.connect(user='vishal.kumar@scale.com',
                                 account='pxa65918',
                                 authenticator='externalbrowser',
                                 warehouse='BOOTCAMP_WH',
                                 database='SCALE_PLAYPEN',
                                 role='SNOWFLAKE_USERS')
cs = con.cursor()


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [82]:
#Get table names from the schema
sql = f'''
SHOW TABLES IN SCALE_PLAYPEN.GOLDFINCH_REPLICA;
'''
cs.execute(sql)
rows = cs.fetchall()
column_names = [desc[0] for desc in cs.description]
tdf = pd.DataFrame(rows, columns=column_names)

#copy table names to a list
table_names = []
for i in tdf['name']:
    table_names.append(i)


In [83]:
def get_random_table_name():
    #generate a random table name from the list
    table_name = np.random.choice(table_names)
    return table_name

def get_all_columns(table_name):
    sql = f'''
    SHOW COLUMNS IN SCALE_PLAYPEN.GOLDFINCH_REPLICA.{table_name};
    '''
    cs.execute(sql)
    rows = cs.fetchall()
    column_names = [desc[0] for desc in cs.description]
    cdf = pd.DataFrame(rows, columns=column_names)
    #copy column names to a list
    column_names = []
    for i in cdf['column_name']:
        column_names.append(i)
    return column_names


In [84]:
#call get_random_table_name() to get a random table name
table_name = get_random_table_name()
#print (table_name)
#call get_all_columns(table_name) to get all columns for the table
cdf = get_all_columns(table_name)
#print (column_names)

In [85]:
question_easy = "You are a top finance executive, who is known for asking excellent questions about your business. You are presented with data from table " + table_name + " with columns " + str(cdf) + ". Ask a business question such that the SQL query to answer the question should have only have one of these components HAVING, ORDER BY, LIMIT, JOIN, LIKE, OR, WHERE, GROUP BY. Do not mention any column names or database lingo in the question, the question should be from an executive perspective. Only provide the question, do not mention any additional details."
question_medium = "You are a top finance executive, who is known for asking excellent questions about your business. You are presented with data from table " + table_name + " with columns " + str(cdf) + ". Ask a business question such that the SQL query to answer the question should have one of these components HAVING, ORDER BY, LIMIT, JOIN, LIKE, OR, WHERE and uses up to two aggregations such as SUM, AVERAGE, GROUP BY, COUNT, MIN, MAX. Do not mention any column names or database lingo in the question, the question should be from an executive perspective. Only provide the question, do not mention any additional details."
question_hard = "You are a top finance executive, who is known for asking excellent questions about your business. You are presented with data from table " + table_name + " with columns " + str(cdf) + ". Ask a business question such that the SQL query to answer the question should have one of these components HAVING, ORDER BY, LIMIT, JOIN, LIKE, OR, WHERE and uses two or more aggregations such as SUM, AVERAGE, GROUP BY, COUNT, MIN, MAX. Do not mention any column names or database lingo in the question, the question should be from an executive perspective. Only provide the question, do not mention any additional details."
question_extremely_hard = "You are a top finance executive, who is known for asking excellent questions about your business. You are presented with data from table " + table_name + " with columns " + str(cdf) + ". Ask a business question such that the SQL query to answer the question should have one of these components HAVING, ORDER BY, LIMIT, JOIN, LIKE, OR, WHERE and uses up to two aggregations such as SUM, AVERAGE, GROUP BY, COUNT, MIN, MAX as well as one of these functions EXCEPT, UNION, INTERSECTED, NESTED. Do not mention any column names or database lingo in the question, the question should be from an executive perspective. Only provide the question, do not mention any additional details."

In [87]:
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role":"user", "content":question_extremely_hard}]
)
#parse the response to get the only the "content" part and save it in response variable
response = completion["choices"][0]["message"]["content"]
print (response)


What is the total sales amount for each product category, ordered from highest to lowest?
